In [1]:
import yellowcab.cleaning

In [2]:
cab_df = yellowcab.io.read_parquet('01.parquet')
cab_df

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,2020-04-01 00:41:22,2020-04-01 01:01:53,1.0,1.20,1.0,41,24,2.0,5.5,0.5,0.5,0.00,0.0,0.3,6.80,0.0
1,2020-04-01 00:56:00,2020-04-01 01:09:25,1.0,3.40,1.0,95,197,1.0,12.5,0.5,0.5,2.75,0.0,0.3,16.55,0.0
2,2020-04-01 00:00:26,2020-04-01 00:09:25,1.0,2.80,1.0,237,137,1.0,10.0,3.0,0.5,1.00,0.0,0.3,14.80,2.5
3,2020-04-01 00:24:38,2020-04-01 00:34:38,0.0,2.60,1.0,68,142,1.0,10.0,3.0,0.5,1.00,0.0,0.3,14.80,2.5
4,2020-04-01 00:13:24,2020-04-01 00:18:26,1.0,1.44,1.0,263,74,1.0,6.5,0.5,0.5,3.00,0.0,0.3,13.30,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218475,2020-04-30 23:03:12,2020-04-30 23:11:51,1.0,1.67,1.0,239,50,1.0,8.5,0.5,0.5,3.08,0.0,0.3,15.38,2.5
218476,2020-04-30 23:06:30,2020-04-30 23:16:16,5.0,3.53,1.0,141,4,1.0,12.0,0.5,0.5,2.00,0.0,0.3,19.75,2.5
218477,2020-04-30 23:13:40,2020-04-30 23:18:17,1.0,1.60,1.0,170,140,1.0,6.5,3.0,0.5,5.00,0.0,0.3,15.30,2.5
218478,2020-04-30 23:30:39,2020-04-30 23:42:56,1.0,6.70,1.0,107,168,2.0,20.0,3.0,0.5,0.00,0.0,0.3,23.80,2.5


In [3]:
cab_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218480 entries, 0 to 218479
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   tpep_pickup_datetime   218480 non-null  datetime64[ns]
 1   tpep_dropoff_datetime  218480 non-null  datetime64[ns]
 2   passenger_count        218480 non-null  float64       
 3   trip_distance          218480 non-null  float64       
 4   RatecodeID             218480 non-null  float64       
 5   PULocationID           218480 non-null  int64         
 6   DOLocationID           218480 non-null  int64         
 7   payment_type           218480 non-null  float64       
 8   fare_amount            218480 non-null  float64       
 9   extra                  218480 non-null  float64       
 10  mta_tax                218480 non-null  float64       
 11  tip_amount             218480 non-null  float64       
 12  tolls_amount           218480 non-null  floa

In [4]:
relevant_columns = ['passenger_count', 'trip_distance', 'RatecodeID',
                    'PULocationID', 'DOLocationID', 'payment_type',
                    'fare_amount', 'extra', 'mta_tax', 'tip_amount',
                    'tolls_amount', 'improvement_surcharge', 'total_amount',
                    'congestion_surcharge']

In [5]:
for c in relevant_columns:
    print("---- %s ---" % c)
    print(cab_df[c].value_counts())

---- passenger_count ---
1.0    177793
2.0     19404
0.0      6582
5.0      4642
3.0      4553
6.0      3910
4.0      1595
7.0         1
Name: passenger_count, dtype: int64
---- trip_distance ---
0.00     5526
0.90     4568
0.80     4446
0.70     4424
1.00     4400
         ... 
17.75       1
21.92       1
23.04       1
25.20       1
16.25       1
Name: trip_distance, Length: 2612, dtype: int64
---- RatecodeID ---
1.0     216080
2.0       1025
5.0       1025
4.0        175
3.0        158
99.0        15
6.0          2
Name: RatecodeID, dtype: int64
---- PULocationID ---
140    10361
238    10012
75      9625
263     9228
137     8970
       ...  
172        1
96         1
46         1
245        1
154        1
Name: PULocationID, Length: 243, dtype: int64
---- DOLocationID ---
75     10288
236     8056
140     7995
238     7685
74      7261
       ...  
253        2
111        2
109        2
204        1
99         1
Name: DOLocationID, Length: 255, dtype: int64
---- payment_type ---
1.

In [6]:
x = yellowcab.cleaning.clean_dataset(cab_df, month=1, verbose=True)

--> 6582 invalid entries found in passenger_count
--> 5377 invalid entries found in trip_distance
--> 1140 invalid entries found in fare_amount
--> 0 invalid entries found in extra
--> 0 invalid entries found in mta_tax
--> 0 invalid entries found in tip_amount
--> 0 invalid entries found in tolls_amount
--> 0 invalid entries found in improvement_surcharge
--> 0 invalid entries found in total_amount
--> 0 invalid entries found in congestion_surcharge
13099 invalid entries have been successfully dropped!
130 invalid date entries have been successfully dropped!
--> 3873 outlier detected and removed from passenger_count column using zscore
--> 1759 outlier detected and removed from trip_distance column using zscore
--> 3951 outlier detected and removed from fare_amount column using HBOS algorithm
--> 8985 outlier detected and removed from extra column using HBOS algorithm
--> 205 outlier detected and removed from mta_tax column using HBOS algorithm
--> 310 outlier detected and removed fro

In [7]:
x

,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,...,total_amount,congestion_surcharge,trip_duration_minutes,pickup_month,pickup_day,pickup_hour,dropoff_month,dropoff_day,dropoff_hour,weekend
0,2020-04-01 00:41:22,2020-04-01 01:01:53,1.0,1.20,Standard rate,41,24,Cash,5.5,0.5,...,6.80,0.0,20.516667,4,1,0,4,1,1,False
1,2020-04-01 00:56:00,2020-04-01 01:09:25,1.0,3.40,Standard rate,95,197,Credit card,12.5,0.5,...,16.55,0.0,13.416667,4,1,0,4,1,1,False
2,2020-04-01 00:13:24,2020-04-01 00:18:26,1.0,1.44,Standard rate,263,74,Credit card,6.5,0.5,...,13.30,2.5,5.033333,4,1,0,4,1,0,False
3,2020-04-01 00:24:36,2020-04-01 00:33:09,1.0,2.93,Standard rate,75,170,Cash,10.5,0.5,...,14.30,2.5,8.550000,4,1,0,4,1,0,False
4,2020-04-01 00:56:56,2020-04-01 01:09:13,1.0,6.86,Standard rate,141,243,Credit card,20.0,0.5,...,28.56,2.5,12.283333,4,1,0,4,1,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184165,2020-04-30 23:25:48,2020-04-30 23:34:04,1.0,2.09,Standard rate,186,142,Credit card,9.0,0.5,...,16.64,2.5,8.266667,4,30,23,4,30,23,False
184166,2020-04-30 23:54:13,2020-05-01 00:02:56,1.0,2.19,Standard rate,239,230,Credit card,9.5,0.5,...,17.91,2.5,8.716667,4,30,23,5,1,0,False
184167,2020-04-30 23:03:12,2020-04-30 23:11:51,1.0,1.67,Standard rate,239,50,Credit card,8.5,0.5,...,15.38,2.5,8.650000,4,30,23,4,30,23,False
184168,2020-04-30 23:06:30,2020-04-30 23:16:16,5.0,3.53,Standard rate,141,4,Credit card,12.0,0.5,...,19.75,2.5,9.766667,4,30,23,4,30,23,False


In [8]:
# how many faulty data do we still got here?
for c in relevant_columns:
    print("---- %s ---" % c)
    print(x[c].value_counts())

---- passenger_count ---
1.0    156364
2.0     17665
5.0      4403
3.0      4250
4.0      1488
Name: passenger_count, dtype: int64
---- trip_distance ---
0.90     3939
0.80     3820
0.70     3819
1.00     3677
1.10     3482
         ... 
11.48       1
11.93       1
12.14       1
11.92       1
12.17       1
Name: trip_distance, Length: 1214, dtype: int64
---- RatecodeID ---
Standard rate         184066
Negotiated fare           58
Nassau/Westchester        45
Newark                     1
Name: RatecodeID, dtype: int64
---- PULocationID ---
238    8967
140    8591
75     8545
263    8171
236    7718
       ... 
154       1
9         1
180       1
131       1
6         1
Name: PULocationID, Length: 231, dtype: int64
---- DOLocationID ---
75     9261
236    7238
140    7183
238    6776
74     6463
       ... 
253       1
154       1
245       1
150       1
175       1
Name: DOLocationID, Length: 238, dtype: int64
---- payment_type ---
Credit card    111234
Cash            70975
No charge  